# Fairness in Biometrics: a figure of merit to assess biometric verification

This notebook is an extension of the paper `Fairness in Biometrics: a figure of merit to assess biometric verification`.
The goal is to give a little bit more intuition about the issues with respect to fairness in biometric verification and how the Fairness Discrepancy Rate can be usefull.

## Fair example

The cell below presents the fair example from section 3.1
Play with the variables below to sense the problematic of having one decision threshold that is fair to **all demographic groups**.

In [1]:
%matplotlib widget

#################################################
## PLAYGROUND: PLAY WITH THE VALUES OF VARIABLES BELOW TO CHANGE THE BEHAVIOUR OF THE FAIR VERIFICATION SYSTEM

# RANGE OF DECISION THRESHOLDS USED IN THE DEVELOPMENT SET
TEN_POWER_OF_X = [0.1, 0.01, 0.001, 0.0001, 0.00001] # BASICALLY, THE FMR THRESHOLDS

# NUMBER OF IDENTITIES PER DEMOGRAPHIC
IDENTITIES_PER_DEMOGRAPHIC = 100

# REPRESENTS HOW A SAMPLES ARE SAMPLED
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE AVERAGE EUCLIDEAN DISTANCE BETWEEN EACH IDENTITY
# EACH IDENTITY IS SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_MEAN_DISTANCE = [1, 1, 1]
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE STANDARD DEVIATIONS OF THE SAMPLES SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_STD = [0.2, 0.2, 0.2]
#################################################



import plots
import experiments
import scoring
import fairness_metric
import numpy as np


# fetching the samples
impostors_per_demographic_dev_fair, genuines_per_demographic_dev_fair, \
impostors_per_demographic_eval_fair, genuines_per_demographic_eval_fair,_ = experiments.generate_abstract_experiment(
    DEMOGRAPHIC_DISPLACEMENTS = DEMOGRAPHIC_MEAN_DISTANCE,
    DEMOGRAPHIC_STD = DEMOGRAPHIC_STD,    
    FAR_THRESHOLDS=TEN_POWER_OF_X,
    DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC=IDENTITIES_PER_DEMOGRAPHIC
)

n_demographics_fair = len(DEMOGRAPHIC_MEAN_DISTANCE)

# computing TAU USING THE ZEROTH-EFFORT IMPOSTORS from the development set
taus_fair = scoring.compute_thresholds(impostors_per_demographic_dev_fair, TEN_POWER_OF_X)

# Plot Figure 1 using data from evaluation set
plots.plot_cohort_box_plot(impostors_per_demographic_eval_fair, 
                     genuines_per_demographic_eval_fair, 
                     [f"{i}" for i in range(n_demographics_fair)],
                     thresholds=taus_fair,
                     far_thresholds=[10**(i+1) for i,_ in enumerate(TEN_POWER_OF_X)]
                    )



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Unfair example

The cell below presents the UNfair example from section 3.1
Play with the variables below to sense the problematic of having one decision threshold that is fair to **all demographic groups**.


In [4]:
#################################################
## PLAYGROUND: PLAY WITH THE VALUES OF VARIABLES BELOW TO CHANGE THE BEHAVIOUR OF THE FAIR VERIFICATION SYSTEM

# NUMBER OF IDENTITIES PER DEMOGRAPHIC
IDENTITIES_PER_DEMOGRAPHIC = 100

# REPRESENTS HOW A SAMPLES ARE SAMPLED
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE AVERAGE EUCLIDEAN DISTANCE BETWEEN EACH IDENTITY
# EACH IDENTITY IS SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_MEAN_DISTANCE = [0.8, 1.3, 1.5]
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE STANDARD DEVIATIONS OF THE SAMPLES SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_STD = [0.1, 0.2, 0.4]
#################################################

n_demographics_unfair = len(DEMOGRAPHIC_MEAN_DISTANCE)

impostors_per_demographic_dev_unfair, genuines_per_demographic_dev_unfair, \
impostors_per_demographic_eval_unfair, genuines_per_demographic_eval_unfair, _ = \
experiments.generate_abstract_experiment(DEMOGRAPHIC_DISPLACEMENTS=DEMOGRAPHIC_MEAN_DISTANCE,
                                        DEMOGRAPHIC_STD=DEMOGRAPHIC_STD,
                                        DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC=IDENTITIES_PER_DEMOGRAPHIC)
                       
# computing TAU USING THE ZEROTH-EFFORT IMPOSTORS from the development set
taus_unfair = scoring.compute_thresholds(impostors_per_demographic_dev_unfair, TEN_POWER_OF_X)

# Plot Figure 1 using data from evaluation set
plots.plot_cohort_box_plot(impostors_per_demographic_eval_unfair,
                           genuines_per_demographic_eval_unfair,
                           [f"{i}" for i in range(n_demographics_unfair)],
                           thresholds=taus_unfair,
                           far_thresholds=[10**(i+1) for i,_ in enumerate(taus_unfair)])



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fairness Discrepancy Curve

It computes **FDR** for the synthetic fair and unfair systems and computes the Area Under FDR.


In [5]:
import fairness_metric

### FAIR
f_tau_fair = fairness_metric.compute_fairness_biometric_verification(impostors_per_demographic_eval_fair, 
                                                                     genuines_per_demographic_eval_fair,
                                                                     taus_fair,
                                                                     len(taus_fair),
                                                                     n_demographics_fair)

f_tau_unfair = fairness_metric.compute_fairness_biometric_verification(impostors_per_demographic_eval_unfair,
                                                                       genuines_per_demographic_eval_unfair,
                                                                       taus_unfair,
                                                                       len(taus_unfair),
                                                                       n_demographics_unfair)

fairness_metric.fairness_plot_clean([f_tau_fair,f_tau_unfair],
                                    labels=["Fair Biom. System","Unfair Biom. System"],
                                    far_thresholds=TEN_POWER_OF_X,
                                    title="")



print(f"AU-FPR from fair system {fairness_metric.fpr_auc_score(f_tau_fair, TEN_POWER_OF_X)}")
print(f"AU-FPR from unfair system {fairness_metric.fpr_auc_score(f_tau_unfair, TEN_POWER_OF_X)}")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AU-FPR from fair system 0.9992738830317849
AU-FPR from unfair system 0.660289730763997


# The limits of fairness